Installing required modules

In [ ]:
import sys
import subprocess

def install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg]);

packages = ["torch", "torchvision", "seaborn", "psutil"];

for p in packages:
    try:
        __import__(p.split("-")[0]);
    except ImportError:
        print("Installing package:", p);
        install(p);

importing modules and basic setup

In [ ]:
import os;
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from data_loader import VideoDataset2D
from models import ResNet18Temporal
from utils import train_one_epoch, eval_one_epoch, EarlyStopping
from tqdm import trange
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time
import random
import psutil
from data_loader import VideoDataset3D
from models import ResNet3D

from sklearn.metrics import precision_recall_fscore_support
import json

### Implementing 2D-CNN

Reproducibility and Random Seed Initialization

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Creating required directories

In [ ]:

os.makedirs("../dataset_info/sample_frames", exist_ok=True);
os.makedirs("../results", exist_ok=True);
os.makedirs("../results/confusion_matrices", exist_ok=True);
os.makedirs("../results/performance_plots", exist_ok=True);
os.makedirs("../results/feature_visualizations", exist_ok=True);
os.makedirs("../results/saved_models", exist_ok=True);
os.makedirs("../results/saved_feature_matrices", exist_ok=True);

Directory path for saving trained models

In [ ]:

SAVE_DIR = "../results/saved_models"
os.makedirs(SAVE_DIR, exist_ok=True)

MODEL_2D_PATH = os.path.join(SAVE_DIR, "best_resnet18_temporal.pth")
MODEL_3D_PATH = os.path.join(SAVE_DIR, "best_resnet3d.pth")

Transforms

In [ ]:
train_tfms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_tfms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])


Dataset & loaders

In [ ]:
class_map = {"PullUps": 0, "Punch": 1, "PushUps": 2}
num_classes = len(class_map)

dataset_root = "../dataset_info/dataset"

train_ds_2d = VideoDataset2D("../dataset_info/dataset/splits/train.csv",
                         dataset_root, class_map,
                         num_frames=20, transform=train_tfms, train=True)

val_ds_2d = VideoDataset2D("../dataset_info/dataset/splits/val.csv",
                       dataset_root, class_map,
                       num_frames=20, transform=val_tfms, train=False)

test_ds_2d = VideoDataset2D("../dataset_info/dataset/splits/test.csv",
                        dataset_root, class_map,
                        num_frames=20, transform=val_tfms, train=False)

train_loader_2d = DataLoader(train_ds_2d, batch_size=4, shuffle=True, num_workers=4)
val_loader_2d   = DataLoader(val_ds_2d, batch_size=4, shuffle=False, num_workers=4)
test_loader_2d  = DataLoader(test_ds_2d, batch_size=4, shuffle=False, num_workers=4)


Model, optimizer, scheduler

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_2d = ResNet18Temporal(num_classes=num_classes, pooling="avg", dropout=0.5).to(device)


criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.9)

optimizer = torch.optim.Adam(model_2d.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

early_stop = EarlyStopping(patience=3)


Training loop (with early stopping) 
- This will take around 30 minutes to complete.

In [ ]:
EPOCHS = 12
best_val_acc = 0

train_losses = []
val_losses = []
train_accs = []
val_accs = []


# ================= LOAD IF EXISTS =================
if os.path.exists(MODEL_2D_PATH):

    print("Found saved 2D model. Loading...")

    model_2d.load_state_dict(torch.load(MODEL_2D_PATH, map_location=device))
    model_2d.to(device)
    model_2d.eval()

else:
    print("No saved 2D model found. Starting training...")


    for epoch in trange(EPOCHS, desc="2D CNN Epochs"):

        train_loss, train_acc = train_one_epoch(
            model_2d, train_loader_2d, optimizer, criterion, device)

        val_loss, val_acc = eval_one_epoch(
            model_2d, val_loader_2d, criterion, device)

        scheduler.step()

        print(f"\nEpoch [{epoch+1}/{EPOCHS}] "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} || "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model_2d.state_dict(), MODEL_2D_PATH)

        early_stop(val_loss)
        if early_stop.stop:
            print("⏹ Early stopping triggered")
            break

    print("2D CNN Training complete. Best Val Acc:", best_val_acc)


Loading best model

In [ ]:
# Load best saved model
model_2d.load_state_dict(torch.load(MODEL_2D_PATH, map_location=device))
model_2d.eval()

print("Model loaded successfully")
print("Backbone: ResNet-18 (ImageNet pretrained)")
print("Input: fixed-length RGB frame sequences (224x224)")
print("Temporal aggregation: Temporal pooling")
print("Classifier head: Dropout → Fully Connected (3 classes)")

print("Trainable parameters:",
      sum(p.numel() for p in model_2d.parameters() if p.requires_grad))

print("Total parameters:",
      sum(p.numel() for p in model_2d.parameters()))



Test set evaluation

In [ ]:
all_preds = []
all_labels = []

with torch.no_grad():
    for x, y in test_loader_2d:
        x, y = x.to(device), y.to(device)
        out = model_2d(x)
        preds = torch.argmax(out, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)


Evaluation metrices

In [ ]:
print("Test Accuracy:", accuracy_score(all_labels, all_preds))

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=list(class_map.keys())))

Confusion matrix

In [ ]:
conf_matrix = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(6,5))
sns.heatmap(conf_matrix, annot=True, fmt="d",
            xticklabels=class_map.keys(),
            yticklabels=class_map.keys(),
            cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix – 2D CNN (ResNet-18)")
plt.savefig("../results/confusion_matrices/2d_resnet18_confusion_matrix.png")
plt.show()
plt.close()


Learning curves

In [ ]:
plt.figure()
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss Curve – 2D CNN")
plt.legend()
plt.savefig("../results/performance_plots/2dcnn_loss_curve.png", dpi=300, bbox_inches="tight");
plt.show()
plt.close()

plt.figure()
plt.plot(train_accs, label="Train Accuracy")
plt.plot(val_accs, label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy Curve – 2D CNN")
plt.legend()
plt.savefig("../results/performance_plots/2dcnn_accuracy_curve.png", dpi=300, bbox_inches="tight");
plt.show()
plt.close()

Inference time (computational analysis)

In [ ]:
x, y = next(iter(test_loader_2d))
x = x.to(device)

start = time.time()
with torch.no_grad():
    _ = model_2d(x)
end = time.time()

inf_time_2d = (end - start) / x.size(0)  

print("Inference time per batch (seconds):", end-start)
print("Approx inference time per video (seconds)):", inf_time_2d)


Model size (parameter count)

In [ ]:
total_params_2d = sum(p.numel() for p in model_2d.parameters())
trainable_params_2d = sum(p.numel() for p in model_2d.parameters() if p.requires_grad)

print("Total parameters:", total_params_2d)
print("Trainable parameters:", trainable_params_2d)


Error analysis

In [ ]:
wrong_idx = np.where(all_preds != all_labels)[0]
print("Wrong predictions:", len(wrong_idx))

if len(wrong_idx) > 0:
    for i in wrong_idx[:5]:

        row = test_ds_2d.data.iloc[i]   # CSV row
        video_path = row["clip_path"]

        true_label = list(class_map.keys())[all_labels[i]]
        pred_label = list(class_map.keys())[all_preds[i]]

        print("Video:", video_path)
        print("True:", true_label, "| Pred:", pred_label)
        print("-"*50)


File size on disk

In [ ]:
model_path = MODEL_2D_PATH
file_size_mb_2d = os.path.getsize(model_path) / (1024 * 1024)

print("Saved model file size: {:.2f} MB".format(file_size_mb_2d))


Memory usage

In [ ]:
print("CUDA available:", torch.cuda.is_available())
print("Torch CUDA version:", torch.version.cuda)
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


if torch.cuda.is_available():
    print("GPU memory allocated:",
          torch.cuda.memory_allocated() / (1024**2), "MB")
    print("GPU memory reserved:",
          torch.cuda.memory_reserved() / (1024**2), "MB")

process = psutil.Process(os.getpid())
ram_mb = process.memory_info().rss / (1024 * 1024)

print("Current RAM usage:", ram_mb, "MB")


Saving the results for comparitive analyisis

In [ ]:
acc2d = accuracy_score(all_labels, all_preds)
p2d, r2d, f12d, _ = precision_recall_fscore_support(all_labels, all_preds, average="macro")

stats_2d = {
    "accuracy": acc2d,
    "precision": p2d,
    "recall": r2d,
    "f1": f12d,
    "inference_time": inf_time_2d,
    "params": total_params_2d,
    "model_size_mb": file_size_mb_2d
}

os.makedirs("../results", exist_ok=True)
with open("../results/stats_2d.json", "w") as f:
    json.dump(stats_2d, f, indent=4)


### Implementing 3D CNN - I3D (Resnet 3d)

3D Dataloaders

In [ ]:
train_ds_3d = VideoDataset3D("../dataset_info/dataset/splits/train.csv", dataset_root, class_map,
                             num_frames=16, transform=train_tfms, train=True)

val_ds_3d   = VideoDataset3D("../dataset_info/dataset/splits/val.csv", dataset_root, class_map,
                             num_frames=16, transform=val_tfms, train=False)

test_ds_3d  = VideoDataset3D("../dataset_info/dataset/splits/test.csv", dataset_root, class_map,
                             num_frames=16, transform=val_tfms, train=False)

train_loader_3d = DataLoader(train_ds_3d, batch_size=2, shuffle=True, num_workers=4)
val_loader_3d   = DataLoader(val_ds_3d, batch_size=2, shuffle=False, num_workers=4)
test_loader_3d  = DataLoader(test_ds_3d, batch_size=2, shuffle=False, num_workers=4)


x3d, _ = next(iter(test_loader_3d))
print(x3d.shape)

Model, loss, optimizer

In [ ]:
class_map = {"PullUps": 0, "Punch": 1, "PushUps": 2}
num_classes = len(class_map)


device = "cuda" if torch.cuda.is_available() else "cpu"

model_3d = ResNet3D(num_classes=num_classes, dropout=0.5, freeze_backbone=True).to(device)

criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.9)
optimizer = torch.optim.Adam(model_3d.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

early_stop = EarlyStopping(patience=4)

out = model_3d(x3d.to(device))
print(out.shape)

Training loop
- This will take around 30 minutes to complete

In [ ]:
EPOCHS = 12
best_val_acc = 0

train_losses_3d, val_losses_3d = [], []
train_accs_3d, val_accs_3d = [], []


# ================= LOAD IF EXISTS =================
if os.path.exists(MODEL_3D_PATH):

    print("Found saved 3D model. Loading...")

    model_3d.load_state_dict(torch.load(MODEL_3D_PATH, map_location=device))
    model_3d.to(device)
    model_3d.eval()

else:
    print("No saved model found. Starting training...")


    for epoch in trange(EPOCHS, desc="3D CNN Epochs"):

        train_loss, train_acc = train_one_epoch(
            model_3d, train_loader_3d, optimizer, criterion, device)

        val_loss, val_acc = eval_one_epoch(
            model_3d, val_loader_3d, criterion, device)

        scheduler.step()

        print(f"\nEpoch [{epoch+1}/{EPOCHS}] "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} || "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        train_losses_3d.append(train_loss)
        val_losses_3d.append(val_loss)
        train_accs_3d.append(train_acc)
        val_accs_3d.append(val_acc)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model_3d.state_dict(), MODEL_3D_PATH)

        early_stop(val_loss)
        if early_stop.stop:
            print("⏹ Early stopping triggered")
            break

    print("3D CNN Training complete. Best Val Acc:", best_val_acc)



Test evaluation

In [ ]:
model_3d.load_state_dict(torch.load(MODEL_3D_PATH, map_location=device))
model_3d.eval()

all_preds_3d, all_labels_3d = [], []

with torch.no_grad():
    for x, y in test_loader_3d:
        x, y = x.to(device), y.to(device)
        out = model_3d(x)
        preds = torch.argmax(out, dim=1)

        all_preds_3d.extend(preds.cpu().numpy())
        all_labels_3d.extend(y.cpu().numpy())



Metrics + confusion matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("3D CNN Test Accuracy:", accuracy_score(all_labels_3d, all_preds_3d))
print(classification_report(all_labels_3d, all_preds_3d, target_names=class_map.keys()))

cm3d = confusion_matrix(all_labels_3d, all_preds_3d)

plt.figure(figsize=(6,5))
sns.heatmap(cm3d, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_map.keys(),
            yticklabels=class_map.keys())
plt.title("Confusion Matrix – 3D CNN")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

Learning curves (3D)

In [ ]:
plt.figure()
plt.plot(train_losses_3d, label="Train Loss")
plt.plot(val_losses_3d, label="Val Loss")
plt.title("3D CNN Loss Curve")
plt.legend()
plt.show()

plt.figure()
plt.plot(train_accs_3d, label="Train Acc")
plt.plot(val_accs_3d, label="Val Acc")
plt.title("3D CNN Accuracy Curve")
plt.legend()
plt.show()


Inference time

In [ ]:
x, y = next(iter(test_loader_3d))
x = x.to(device)

start = time.time()
with torch.no_grad():
    _ = model_3d(x)
end = time.time()

inf_time_3d = (end - start) / x.size(0)  

print("Inference time per batch:", end-start)
print("Approx inference time per video:", inf_time_3d)


Model size (parameter count)

In [ ]:
total_params_3d = sum(p.numel() for p in model_3d.parameters())
trainable_params_3d = sum(p.numel() for p in model_3d.parameters() if p.requires_grad)

print("3D CNN Total parameters:", total_params_3d)
print("3D CNN Trainable parameters:", trainable_params_3d)


Error Analysis

In [ ]:
wrong_idx = np.where(np.array(all_preds_3d) != np.array(all_labels_3d))[0]
print("Total wrong predictions (3D CNN):", len(wrong_idx))

if len(wrong_idx) > 0:
    #for i in wrong_idx[:5]:
    for i in wrong_idx:    
        row = test_ds_3d.data.iloc[i]
        video_path = row["clip_path"]

        true_label = list(class_map.keys())[all_labels_3d[i]]
        pred_label = list(class_map.keys())[all_preds_3d[i]]

        print("Video:", video_path)
        print("True:", true_label, "| Pred:", pred_label)
        print("-"*50)


Saved model file size (disk storage)

In [ ]:
model_path = MODEL_3D_PATH
file_size_mb_3d = os.path.getsize(model_path) / (1024 * 1024)

print("3D CNN Saved model size: {:.2f} MB".format(file_size_mb_3d))

Memory Usage

In [ ]:
if torch.cuda.is_available():
    print("GPU memory allocated:",
          torch.cuda.memory_allocated() / (1024**2), "MB")
    print("GPU memory reserved:",
          torch.cuda.memory_reserved() / (1024**2), "MB")
else:
    print("GPU not available. Running on CPU.")

process = psutil.Process(os.getpid())
ram_mb = process.memory_info().rss / (1024 * 1024)

print("Current CPU RAM usage: {:.2f} MB".format(ram_mb))


Saving the results

In [ ]:
acc3d = accuracy_score(all_labels_3d, all_preds_3d)
p3d, r3d, f13d, _ = precision_recall_fscore_support(all_labels_3d, all_preds_3d, average="macro")

stats_3d = {
    "accuracy": acc3d,
    "precision": p3d,
    "recall": r3d,
    "f1": f13d,
    "inference_time": inf_time_3d,
    "params": total_params_3d,
    "model_size_mb": file_size_mb_3d
}

with open("../results/stats_3d.json", "w") as f:
    json.dump(stats_3d, f, indent=4)
